In [55]:
# allows editing aoc_utils "live" without restarting kernel
# see https://ipython.org/ipython-doc/stable/config/extensions/autoreload.html
# and https://stackoverflow.com/a/17551284
%load_ext autoreload
%autoreload 2

# Add the aoc_utils path
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import aoc_utils
get_input = aoc_utils.get_input
print = aoc_utils.debug_print

timer = aoc_utils.start_timer()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [56]:
# Useful imports
import re
from collections import defaultdict, deque
import heapq
import functools
import queue
import itertools
import math
import random
from collections import Counter
import statistics
import parse
import operator
from functools import reduce

In [57]:
# aliases from utils
getnums = aoc_utils.getnums

In [58]:
def Day1(data=get_input(1, 2021)):
  nums = list(map(int, data))
  def p1():
    return len([i for i in range(1, len(nums)) if nums[i] > nums[i-1]])
  def p2():
    sums = [sum(nums[i:i+3]) for i in range(len(nums) - 2)]
    return len([i for i in range(1, len(sums)) if sums[i] > sums[i-1]])
    
  return p1(),p2()

assert Day1() == (1548, 1589)

In [59]:
def Day2(data=get_input(2,2021)):
  def p1():
    h,d=0,0
    for line in data:
      n = getnums(line)[0]
      if 'forward' in line:
        h += n
      elif 'down' in line:
        d += n
      else:
        d -= n
    return d * h
  def p2():
    h,d,aim=0,0,0
    for line in data:
      n = getnums(line)[0]
      if 'forward' in line:
        h += n
        d += aim * n
      elif 'down' in line:
        aim += n
      else:
        aim -= n
    return d * h
  return p1(),p2()

assert Day2() == (1694130, 1698850445),Day2()

In [60]:
def Day3(data = get_input(3,2021)):
  b2d = aoc_utils.bin_to_decimal

  def p1():
    cnts = [Counter() for _ in range(12)]
    for line in data:
      for idx,c in enumerate(line):
        cnts[idx].update(c)
    a = b2d(''.join([cnt.most_common()[0][0] for cnt in cnts]))
    b = b2d(''.join([cnt.most_common()[-1][0] for cnt in cnts]))
    return a*b

  def p2():
    def winnow(arr, most_com=True):
      for idx in range(len(arr[0])):
        cnt = Counter()
        for l in arr:
          cnt.update(l[idx])
        v = None
        if most_com:
          v = '0' if cnt['0'] > cnt['1'] else '1'
        else:
          v = '1' if cnt['1'] < cnt['0'] else '0'
        arr = [l for l in arr if l[idx] == v]
        if len(arr) == 1:
          return arr[0]

    O = winnow(data, most_com=True)
    C  = winnow(data, most_com=False)
    return b2d(O)*b2d(C)
  return p1(),p2()

assert Day3() == (1540244, 4203981)

In [61]:
def Day4(data=get_input(4,2021)):
  nums = getnums(data[0])
  boards = list(aoc_utils.split_list(data[2:]))
  grids = []
  for board in boards:
    grid = {}
    for y,line in enumerate(board):
      for x,num in enumerate(getnums(line)):
        grid[(x,y)] = num
    grids.append(grid)

  def boardsum(g, seen):
    unmarked = []
    for x in range(5):
      for y in range(5):
        n = g[(x,y)]
        if n not in seen:
          unmarked.append(n)
    return sum(unmarked)

  def winner(grid, seen):
    for x in range(5):
      if all(grid[(x,y)] in seen for y in range(5)):
        return True
    for y in range(5):
      if all(grid[(x,y)] in seen for x in range(5)):
        return True
    return False

  def p1():
    seen = set()
    for n in nums:
      seen.add(n)
      for g in grids:
        if winner(g, seen):
          return boardsum(g,seen)*n
  def p2():
    seen = set()
    winners = []
    remaining = grids
    for n in nums:
      seen.add(n)
      for g in remaining:
        if winner(g, seen):
          winners.append(g)
        if len(winners) == len(grids):
          return boardsum(g,seen)*n
      remaining = [_g for _g in remaining if _g not in winners]
  return p1(),p2()
assert Day4() == (71708, 34726)

In [62]:
def Day5(data=get_input(5,2021)):
  lines = list(map(getnums, data))
  def points(x1,y1,x2,y2):
    if y1 == y2:
      x1,x2 = sorted([x1,x2])
      return [(x,y1) for x in range(x1,x2+1)]
    elif x1 == x2:
      y1,y2 = sorted([y1,y2])
      return [(x1,y) for y in range(y1,y2+1)]
    else:
      m,b = statistics.linear_regression([x1,x2],[y1,y2])
      assert m == 1 or m == -1
      x1,x2 = sorted([x1,x2])
      return [(x,int(m*x+b)) for x in range(x1,x2+1)]

  assert points(0,0,2,0) == [(0,0), (1,0), (2,0)]
  assert points(0,0,0,2) == [(0,0), (0,1), (0,2)]
  assert points(2,0,0,0) == [(0,0), (1,0), (2,0)]
  assert points(0,2,0,0) == [(0,0), (0,1), (0,2)]
  assert points(1,1,1,3) == [(1,1),(1,2),(1,3)]
  assert sorted(points(9,7,7,7)) == [(7,7),(8,7),(9,7)]
  assert sorted(points(7,7,9,7)) == [(7,7),(8,7),(9,7)]
  assert sorted(points(0,0,3,3)) == [(0,0),(1,1),(2,2),(3,3)]

  p1_cnt = defaultdict(int)
  p2_cnt = defaultdict(int)
  for line in lines:
    x1,y1,x2,y2 = line
    for p in points(x1,y1,x2,y2):
      p2_cnt[p] += 1
      if x1 == x2 or y1 == y2:
        p1_cnt[p] += 1
  p1 = sum([1 for v in p1_cnt.values() if v > 1])
  p2 = sum([1 for v in p2_cnt.values() if v > 1])

  return p1,p2

assert Day5() == (5608, 20299)


In [63]:
def Day6(data = get_input(6, 2021)):
  p1 = 0
  p2 = 0
  data = getnums(data[0])
  P1_DAYS = 80
  P2_DAYS = 256
  cnt = Counter(data)
  for d in range(P2_DAYS):
    cyc = d%7
    tmp = cnt[7]
    cnt[7] = cnt[8]
    cnt[8] = cnt[cyc]
    cnt[cyc] += tmp
    if d + 1 == P1_DAYS:
      p1 = cnt.total()
  p2 = cnt.total()
  return p1,p2

assert Day6() == (360610, 1631629590423) 

In [64]:
def Day7(data = aoc_utils.get_input(7, 2021)):
  p1 = p2 = 0
  data = getnums(data[0])
  median = statistics.median(data)
  for d in data:
    p1 += abs(d - median)

  @functools.lru_cache()
  def get_cost(v,target):
    n = abs(v-target)
    return n * (n+1) / 2

  p2 = min([sum([get_cost(c,target) for c in data]) for target in range(min(data), max(data))])

  return p1,p2

assert Day7() == (347509, 98257206)

In [65]:
def Day8(data=get_input(8,2021)):
  p1 = p2 = 0
  data = get_input(8, 2021)
  for line in data:
    digits = line.split('| ')[1].split(' ')
    for d in digits:
      if len(d) == 7 or len(d) == 4 or len(d) == 3 or len(d) == 2:
        p1 += 1
    pass

  DIGNUMS = [
    [0,1,2,3,4,6], #0
    [1,2], #1
    [0,1,3,4,5], # 2
    [0,1,2,3,5], # 3
    [1,2,5,6], # 4
    [0,2,3,5,6], # 5
    [0,2,3,4,5,6], # 6
    [0,1,2], #7
    [0,1,2,3,4,5,6], # 8
    [0,1,2,3,5,6] # 9
  ]

  def prune(_map):
    found_keys = [k for k,v in _map.items() if len(v) == 1]
    found_vals = [list(v)[0] for k,v in _map.items() if k in found_keys]
    for k in _map.keys():
      if k not in found_keys:
        _map[k].difference_update(found_vals)
    return _map

  def solve_num(s, poss):
    ons = [list(v)[0] for c,v in poss.items() if c in s]
    ons = list(sorted(ons))
    assert ons in DIGNUMS, (ons,s)
    return DIGNUMS.index(ons)

  for line in data:
    signals, digits = line.split(' | ')
    signals = signals.split(' ')
    digits = digits.split(' ')
    poss = defaultdict(lambda: set([0,1,2,3,4,5,6]))
    for s in signals:
      if len(s) == 2:
        _poss = set(DIGNUMS[1]) # 1l
        for c in s: poss[c] = poss[c] & _poss
      elif len(s) == 3:
        _poss = set(DIGNUMS[7]) # 7
        for c in s: poss[c] = poss[c] & _poss
      elif len(s) == 4:
        _poss = set(DIGNUMS[4]) # 4
        for c in s: poss[c] = poss[c] & _poss
      elif len(s) == 7:
        _poss = set(DIGNUMS[8]) # 8
        for c in s: poss[c] = poss[c] & _poss
      elif len(s) == 5:
        _poss = set(DIGNUMS[2]) #2
        _poss = _poss | set(DIGNUMS[3]) #3
        _poss = _poss | set(DIGNUMS[5]) # 5
        for c in s: poss[c] = poss[c] & _poss
      elif len(s) == 6:
        _poss = set(DIGNUMS[6]) # 6
        _poss = _poss | set(DIGNUMS[0]) # 0
        _poss = _poss | set(DIGNUMS[9]) # 9
        for c in s: poss[c] = poss[c] & _poss
      else:
        assert False, s
    found = []
    right_segs = [c for c,v in poss.items() if len(v) == 2]
    assert len(right_segs) == 2
    a,b = right_segs
    a_cnt = len([s for s in signals if a in s])
    b_cnt = len([s for s in signals if b in s])
    if a_cnt == 8:
      assert b_cnt == 9
      poss[a] = set([1])
      poss[b] = set([2])
    else:
      assert a_cnt == 9
      assert b_cnt == 8
      poss[a] = set([2])
      poss[b] = set([1])
    poss = prune(poss)
    left_segs = [c for c,v in poss.items() if len(v) == 2]
    assert len(left_segs) == 2
    a,b = left_segs
    a_cnt = len([s for s in signals if a in s])
    b_cnt = len([s for s in signals if b in s])
    assert poss[a] == set([5,6])
    assert poss[b] == set([5,6])
    if a_cnt == 6:
      assert b_cnt == 7
      poss[a] = set([6])
      poss[b] = set([5])
    else:
      assert b_cnt == 6
      assert a_cnt == 7
      poss[a] = set([5])
      poss[b] = set([6])
    poss = prune(poss)
    rem_segs = [c for c,v in poss.items() if len(v) > 1]
    assert len(rem_segs) == 2
    a,b = rem_segs
    a_cnt = len([s for s in signals if a in s])
    b_cnt = len([s for s in signals if b in s])
    if a_cnt == 7:
      assert b_cnt == 4
      poss[a] = set([3])
      poss[b] = set([4])
    else:
      assert a_cnt == 4
      assert b_cnt == 7
      poss[a] = set([4])
      poss[b] = set([3])
    assert all([len(v) == 1 for v in poss.values()])
    p2 += int(''.join(map(str, [solve_num(s,poss) for s in digits])))


  return p1,p2

assert Day8() == (387, 986034)

In [70]:
def Day9(data=get_input(9,2021)):
  p1 = p2 = 0
  grid = defaultdict(lambda: 9)
  XMAX = len(data[0])
  YMAX = len(data)
  for y,line in enumerate(data):
    for x,c in enumerate(line):
      grid[(x,y)] = int(c)

  lows = []
  for y in range(YMAX):
    for x in range(XMAX):
      v = grid[(x,y)]
      low = all([grid[n] > v for n in aoc_utils.neighbors((x,y))])
      if low:
        lows.append((x,y))
        p1 += v+1

  basins = []
  for low in lows:
    x,y = low
    v = grid[(x,y)]
    seen = set()
    basin = set()
    neighbors = aoc_utils.neighbors(low)
    while len(neighbors):
      n = neighbors.pop()
      if n not in seen:
        seen.add(n)
        if grid[n] < 9:
          basin.add(n)
          for _n in aoc_utils.neighbors(n):
            neighbors.append(_n)
    basins.append(basin)
  bsizes = list(sorted(map(len, basins)))

  p2 = reduce(operator.mul, bsizes[-3:],1)
  return p1,p2

assert Day9() == (486, 1059300)